In [1]:
# не забудьте установить gym, если он у вас ещё не установлен
# при помощи pip install gym

import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# фиксируем сид
np.random.seed(17)
tf.random.set_seed(17)


Наша игра называется `CartPole-v0`, она как раз демонстрируется в документации к gym: https://gym.openai.com/docs/

In [24]:
env = gym.make("CartPole-v0")  # создадим среду
env.seed(17)  # зафиксируем сид для воспроизводимости

[17]

Мы можем изучить, какие параметры нам передаёт среда в качестве состояния, а какие действия м можем совершать.

In [27]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [29]:
env.action_space

Discrete(2)

В качестве состояния среды нам передаётся 4 числа, а совершать мы можем 2 дискретных действия (соответственно, приложить силу влево или вправо).

Давайте обучим агента в виде полносвязной нейросети с механизмом Actor Critic.

In [32]:
num_inputs = 4  # кол-во параметров состояния среды
num_actions = 2  # кол-во возможных действий

num_hidden = 64

# создадим нейросеть при помощи Functional API
# т.к. нам потребуется 2 выхода

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
common2 = layers.Dense(num_hidden, activation="relu")(common)
action = layers.Dense(num_actions, activation="softmax")(common2)
critic = layers.Dense(1)(common2)

model = keras.Model(inputs=inputs, outputs=[action, critic])


In [34]:
# в качестве оптимизатора будем по стандарту использовать ADAM
optimizer = keras.optimizers.Adam(learning_rate=0.01)

# в качестве функции потерь - Huber loss
# она похожа на MSE, но оканчивается линейрой функцией, а не квадратичной
# это придаст чуть больше стабильности нашему процессу обучения
loss = keras.losses.Huber()

Теперь пришло время заставить нашу модель управлять игрой и обучать её.

В каждом эпизоде мы будем производить один раунд игры, после чего обновлять веса модели в соответствии с историей действий и наград.

Критерием успеха будем считать так называемый `running_reward` - как бы "накопленную" в ходе эпизодов награду (с затуханием). Когда она достигнет 195, будем считать, что модель натренировалась достаточно хорошо.

Приступим:

In [37]:
# зададим discount-фактор для награды
gamma = 0.995

# как долго длится эпизод
max_steps_per_episode = 10000

# минимальная машинная точность
eps = np.finfo(np.float32).eps.item()

# выделим списки для хранения значений
# вероятности действий, критика и награды
action_probs_history = []
critic_value_history = []
rewards_history = []


running_reward = 0
episode_count = 0

while True:
    # сбрасываем состояние к изначальному и записываем его
    state = env.reset()
    episode_reward = 0
    
    # проводим раунд игры, записывая работу нашей нейросети в GradientTape
    with tf.GradientTape() as tape:
        # идём по раунду шагами
        for timestep in range(1, max_steps_per_episode):
            env.render()  # для графического отображения окна с игрой

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # зная состояние, модель должна нам предсказать действие 
            # и поправку от критика
            action_probs, critic_value = model(state)
            
            # поправку критика сразу сохраним
            critic_value_history.append(critic_value[0, 0])

            # выберем действие, исходя из предсказанных вероятностей
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            
            # вероятность выбранного действия сохраняем в историю
            # можно сразу и прологарифмировать, как того требует наш лосс
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # говорим среде выполнить выбранное действие
            # и получаем от неё следующее состояние и награду
            # если игра окончена, среда вернут done=True
            state, reward, done, _ = env.step(action)
            
            # записываем полученную награду
            rewards_history.append(reward)
            
            episode_reward += reward

            if done:
                break

        # пересчитываем running_reward с затуханием
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # награда на каждом шаге игры (то, что нам и должен предсказать критик)
        # определяется как суммарная награда от этого и всех последующих шагов (с затуханием)
        # считаем их все в 1 проход и добавляем в returns
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # нормализуем их для повышения стабильности обучения
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # осталось посчитать значения функций потерь, чтобы затем осуществить шаг градиентного спуска
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # на каждом шаге считаем лосс для actor и critic
            # у первого это просто награда (за вычетом поправки от критика), 
            # умноженная на логарифмическую вероятность действия
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # критик же оканчивается обычной регрессией:
            # он должен уметь предсказывать ожидаемую награду на каждом шаге
            critic_losses.append(
                loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # теперь можем осуществить стандартный шаг градиентного спуска
        # чтобы запустить его из одной точки, сложим все лоссы
        # т.к. при суммировании градиент от каждого слагаемого отправится
        # только в направлении данного слагаемого, это то, что нужно
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # перед следующим эпизодом очистим все списки
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    episode_count += 1
    
    # каждые 10 эпизодов будем выводить running reward
    if episode_count % 10 == 0:
        print(f"Episode: {episode_count}, running reward: {running_reward:.2f}")

    # если running reward стал больше 195, цель достигнута
    if running_reward > 195:
        print(f"Solved at episode {episode_count}!")
        break

# саму среду в конце нужно закрыть
# чтобы рендер прекратился и можно было дальше работать в питоне
env.close()

Episode: 10, running reward: 12.02
Episode: 20, running reward: 27.79
Episode: 30, running reward: 42.17
Episode: 40, running reward: 49.42
Episode: 50, running reward: 51.60
Episode: 60, running reward: 69.63
Episode: 70, running reward: 74.66
Episode: 80, running reward: 80.90
Episode: 90, running reward: 97.25
Episode: 100, running reward: 126.91
Episode: 110, running reward: 156.24
Episode: 120, running reward: 173.80
Episode: 130, running reward: 184.31
Episode: 140, running reward: 185.70
Episode: 150, running reward: 160.40
Episode: 160, running reward: 139.71
Episode: 170, running reward: 143.04
Episode: 180, running reward: 165.11
Episode: 190, running reward: 160.95
Episode: 200, running reward: 167.98
Episode: 210, running reward: 180.83
Episode: 220, running reward: 161.18
Episode: 230, running reward: 125.08
Episode: 240, running reward: 124.88
Episode: 250, running reward: 147.79
Episode: 260, running reward: 167.19
Episode: 270, running reward: 159.85
Episode: 280, runni